In [178]:
import random
from copy import deepcopy
import math
import matplotlib.pyplot as plt
import numpy as np

In [179]:
def load(file):
    cities = []
    with open(file, 'r') as f:
        for line in f:
            data = line.split()
            city_num = int(data[0])-1
            x = float(data[1])
            y = float(data[2])
            demand = float(data[3])
            ready_time = float(data[4])
            due_time = float(data[5])
            
            city = {'city_num' : city_num, 'x' : x, 'y' : y, 'demand' : demand, 'ready_time' : ready_time, 'due_time' : due_time}
            cities.append(city)
            
    return cities   

In [180]:
input1 = load('resources/input_1.txt')
input2 = load('resources/input_2.txt')
input3 = load('resources/input_3.txt')

for x in input1:
    print(x)

{'city_num': 0, 'x': 35.0, 'y': 35.0, 'demand': 0.0, 'ready_time': 0.0, 'due_time': 230.0}
{'city_num': 1, 'x': 41.0, 'y': 49.0, 'demand': 10.0, 'ready_time': 161.0, 'due_time': 171.0}
{'city_num': 2, 'x': 35.0, 'y': 17.0, 'demand': 7.0, 'ready_time': 50.0, 'due_time': 60.0}
{'city_num': 3, 'x': 55.0, 'y': 45.0, 'demand': 13.0, 'ready_time': 116.0, 'due_time': 126.0}
{'city_num': 4, 'x': 55.0, 'y': 20.0, 'demand': 19.0, 'ready_time': 149.0, 'due_time': 159.0}
{'city_num': 5, 'x': 15.0, 'y': 30.0, 'demand': 26.0, 'ready_time': 34.0, 'due_time': 44.0}
{'city_num': 6, 'x': 25.0, 'y': 30.0, 'demand': 3.0, 'ready_time': 99.0, 'due_time': 109.0}
{'city_num': 7, 'x': 20.0, 'y': 50.0, 'demand': 5.0, 'ready_time': 81.0, 'due_time': 91.0}
{'city_num': 8, 'x': 10.0, 'y': 43.0, 'demand': 9.0, 'ready_time': 95.0, 'due_time': 105.0}
{'city_num': 9, 'x': 55.0, 'y': 60.0, 'demand': 16.0, 'ready_time': 97.0, 'due_time': 107.0}
{'city_num': 10, 'x': 30.0, 'y': 60.0, 'demand': 16.0, 'ready_time': 124.0, 

In [181]:
class Individual:
    def __init__(self, data, capacity, num_of_vehicles, num_of_cities, service_time):
        self.num_of_vehicles = num_of_vehicles
        self.num_of_cities = num_of_cities
        self.depot = data[0]
        self.data = data
        self.distance_between_cities = self.calc_all_distances()
        self.capacity = capacity
        self.service_time = service_time
        self.time_penalty = 2
        self.capacity_penalty = 5
        self.tolerance = 1
        self.solution = self.initial_solution_nearest_neighbors()
        self.fitness = self.calc_fitness()

    def __len__(self) -> int:
        return len(self.solution)
    
    def calc_distance(self, city1, city2) -> float:
        distance = math.sqrt((city1["x"] - city2["x"])**2 + (city1["y"] - city2["y"])**2)
        return round(distance, 3)
    
    def calc_all_distances(self) -> [[float]]:
        all_distances = [[0 for _ in range(len(self.data))] for _ in range(len(self.data))]
        
        for i in range(len(self.data)):
            for j in range(len(self.data)):
                if i != j:
                    distance = self.calc_distance(self.data[i], self.data[j])
                    all_distances[i][j] = distance
                else:
                    all_distances[i][j] = 0
                
        return all_distances


    def initial_solution_nearest_neighbors(self):
        routes = [[] for _ in range(self.num_of_vehicles)]
        remaining_cities = [i for i in range(1, self.num_of_cities + 1)]
        
        return self.generate_solution_nearest_neighbors(routes, remaining_cities)
    
    def get_feasible_cities(self, remaining_cities, start_city, current_time, capacity):
            feasible_cities = [(next_city, round(max(self.distance_between_cities[start_city][next_city], 
                                       self.data[next_city]["ready_time"] - current_time - self.service_time), 3)) 
                               for next_city in remaining_cities \
                               if round(current_time + self.distance_between_cities[start_city][next_city] + \
                                        self.service_time, 3) <= 
                                  self.data[next_city]["due_time"] + self.tolerance and \
                                  capacity >= self.data[next_city]["demand"]]
            
            feasible_cities.sort(key = lambda x: x[1])
            return feasible_cities
    
    
    def generate_feasible_routes(self, routes, remaining_cities) -> [[int]]: 
        for route in routes:
            while True:
                if not route:
                    feasible_cities = self.get_feasible_cities(remaining_cities, 0, 0, self.capacity)
                else:
                    _, current_time, remaining_capacity = self.route_fitness(route)
                    feasible_cities = self.get_feasible_cities(remaining_cities, route[-1], current_time, remaining_capacity)

                if not feasible_cities:
                    break
                    
                if random.random() < 0.25:
                    city_index = random.choice(feasible_cities)[0]
                else:
                    city_index = feasible_cities[0][0]

                remaining_cities.remove(city_index)
                route.append(city_index)

                is_unfeasible, _ = self.is_route_unfeasible(route)
                if is_unfeasible:
                    route.pop(-1)
                    remaining_cities.append(city_index)
                    continue
    
        if not remaining_cities:
            return routes
        
        # create an unfeasible route
        for route in routes:
            if not route:
                for city in remaining_cities:
                    route.append(city)
                    is_unfeasible, _ = self.is_route_unfeasible(route)
                    print('is unfeasible:', is_unfeasible)
                    if is_unfeasible:
                        route.pop(-1)
                        remaining_cities.append(city)
                        break 
        
        return routes
        
    def generate_solution_nearest_neighbors(self, routes, remaining_cities) -> [int]:        
        generated_routes = self.generate_feasible_routes(routes, remaining_cities)
        return self.create_solution(generated_routes)

    def create_solution(self, routes) -> [int]:
        solution = []
        for i, route in enumerate(routes):
            for city in route:
                solution.append(city)
            solution.append(self.num_of_cities + i + 1)
        
        # remove empty routes
        for city in reversed(solution):
            if city > 100:
                solution.pop(-1)
            else:
                break
                
        return solution


    def generate_solution_sweep(self):
        pass

    def is_route_unfeasible(self, route) -> (bool, int):
        if not route:
            return False, None
        
        current_time = 0
        previous_city = 0
        remaining_capacity = self.capacity 
        
        for current_city in route:
            current_city_data = self.data[current_city]
            distance = self.distance_between_cities[previous_city][current_city]
            
            if round(current_time + distance + self.service_time, 3) > current_city_data["due_time"] + self.tolerance:
                # print('\ntime violation')
                # print('current time + distance + service time =:', round(current_time + distance + self.service_time, 3))
                # print('city due time + tolerance =', current_city_data["due_time"] + self.tolerance)
                # print('city that produced violation:', current_city)
                return True, current_city 
            
            elif remaining_capacity < current_city_data["demand"]:
                print('\ncapacity violation')
                print('city that produced violation:', current_city)
                return True, current_city
                
            current_time += distance + self.service_time
            previous_city = current_city
            remaining_capacity -= current_city_data["demand"]
            
        # return to depot
        if round(current_time + self.distance_between_cities[previous_city][0] + self.service_time, 3) > \
           self.depot["due_time"] + self.tolerance:
            print('\ntime violation - return to depot')
            print('city that produced violation:', previous_city)
            return True, previous_city
        
        return False, None
    
    def route_fitness(self, route) -> (float, float, float):
        if not route:
            return 0, 0, self.capacity
        
        fitness = 0
        current_time = 0
        previous_city = 0
        remaining_capacity = self.capacity
        
        for current_city in route:
            current_city_data = self.data[current_city]
            distance = self.distance_between_cities[previous_city][current_city]
            
            if round(current_time + distance + self.service_time, 3) > current_city_data["due_time"]:
                fitness += (current_time + distance + self.service_time - current_city_data["due_time"])*self.time_penalty
                
            if remaining_capacity < current_city_data["demand"]:
                fitness += (current_city_data["demand"] - remaining_capacity)*self.capacity_penalty
                
            current_time += distance + self.service_time
            previous_city = current_city
            remaining_capacity -= current_city_data["demand"]
            
        fitness += current_time + self.distance_between_cities[previous_city][0] + self.service_time
        fitness = round(fitness, 3)
        
        return fitness, current_time, remaining_capacity
    
    def get_routes(self) -> [[int]]:
        routes = []
        route = []
        # all routes except for the last route
        for city_index in self.solution:
            if city_index <= self.num_of_cities:
                route.append(city_index)
            else:
                if len(route) > 0:
                    routes.append(route)
                route = []

        # last route
        if len(route) > 0:
            routes.append(route)

        return routes
    
    def calc_fitness(self) -> float:
        routes = self.get_routes()
        
        # more routes than available vehicles
        if sum(1 for r in routes if len(r) > 0) > self.num_of_vehicles:
            return float('inf')

        fitness = 0
        for route in routes:
            if len(route) == 0:
                continue
            f, _, _ = self.route_fitness(route)
            fitness += f
            
        return fitness

    def is_feasible(self) -> bool:
        routes = self.get_routes()
        if sum(1 for r in routes if len(r) > 0) > self.num_of_vehicles:
            print("more vehicles than available violation")
            return False

        for route in routes:
            is_unfeasible, _ = self.is_route_unfeasible(route)
            if is_unfeasible:
                return False
        
        return True

In [182]:
x = Individual(input1, 200, 50, 100, 10)
print(x.solution)
print(x.is_feasible())

y = Individual(input1, 200, 50, 100, 10)
print(y.solution)
print(y.is_feasible())

[80, 93, 85, 43, 13, 58, 101, 23, 75, 89, 37, 91, 100, 102, 32, 11, 50, 26, 1, 70, 103, 59, 15, 73, 22, 56, 4, 77, 104, 62, 94, 99, 6, 25, 105, 92, 95, 3, 79, 34, 35, 106, 5, 20, 9, 66, 107, 72, 7, 46, 17, 60, 108, 42, 84, 61, 55, 109, 27, 28, 69, 88, 18, 10, 48, 110, 45, 44, 86, 97, 96, 111, 33, 12, 76, 81, 54, 24, 112, 14, 57, 87, 68, 113, 39, 21, 40, 53, 74, 114, 83, 30, 51, 115, 2, 98, 16, 116, 63, 64, 90, 117, 31, 52, 19, 8, 118, 47, 49, 119, 36, 120, 82, 78, 121, 65, 71, 122, 29, 67, 123, 38, 41]
True
[95, 2, 32, 10, 1, 77, 101, 59, 83, 61, 85, 58, 93, 91, 100, 102, 92, 98, 17, 99, 94, 97, 37, 13, 89, 103, 5, 82, 50, 3, 25, 80, 104, 72, 9, 81, 4, 105, 42, 15, 73, 22, 26, 24, 106, 27, 28, 38, 43, 74, 107, 66, 30, 51, 34, 35, 70, 108, 33, 12, 76, 79, 20, 48, 109, 96, 16, 56, 55, 54, 110, 8, 46, 19, 49, 60, 111, 45, 44, 84, 112, 14, 53, 6, 68, 113, 39, 41, 57, 114, 63, 62, 88, 115, 31, 69, 7, 18, 116, 78, 29, 117, 47, 11, 90, 118, 36, 64, 119, 52, 21, 40, 87, 120, 67, 23, 121, 65, 7

In [183]:
x.solution == y.solution

False

## Problem: Nedostižni gradovi

Gradovi 14, 36, 63, 92 predstavljaju nedopustiva rešenja.
Nemoguće ih je opslužiti u zadatom vremenskom roku ukoliko se krene od depoa, što je ujedno i početni trenutak svake rute. 

Kako za svaki grad dobijamo vremensko prekoračenje od početne tačke bilo koje rute, zaključujemo da su ti gradovi u potpunosti nedostižni i nije ih moguće obraditi u zadatom vremenskom roku.

## Rešenje: Postoji više mogućnosti

 - Prvo i intuitivno rešenje koje se nameće je da u potpunosti odbacimo navedene gradove i na dalje da posmatramo samo ostale, dostižne gradove koji proizvode dopustiva rešenja.
 
 - Drugo rešenje bi bilo da postavimo prag tolerancije na prekoračenje vremenskog roka. Ukoliko je prekoračeno vreme unutar praga tolerancije tada ćemo rešenje da smatramo dopustivim, ali uz penalizaciju fitness vrednosti.
 
 - Treće rešenje razmatra sve one gradove koji prekoračuju prag tolerancije nad vremenskim ograničenjem. Takva rešenja se smatraju nezadovoljivim i penalizuje se u odnosu na vreme zakašnjenja.

# Selection

In [184]:
def random_selection(population):
    return random.choice(population)

In [185]:
def tournament_selection(population, tournament_size):
    tournament = random.sample(population, tournament_size)
    return max(tournament, key = lambda x: x.fitness)

In [186]:
def roulette_selection(population):
    total_fitness = sum([individual.fitness for individual in population])
    selection_probs = [individual.fitness / total_fitness for individual in population]
    return random.choices(population, weights=selection_probs)[0]

In [187]:
def rang_selection(population):
    n = len(population)
    rang_sum = n * (n+1) / 2
    selection_probs = []
    for rang, _ in enumerate(sorted(population, key=lambda x: x.fitness), 1):
        selection_probs.append(rang / rang_sum)
        
    return random.choices(population, weights=selection_probs)[0]

In [188]:
def selection(param, population):
    if param["tournament_size"] is not None:
        return param["selection"](population, param["tournament_size"])
    else:
        return param["selection"](population)

In [189]:
class Example_Individual:
    def __init__(self, fitness, name):
        self.fitness = fitness
        self.name = name

example_population = [Example_Individual(fitness=0.8, name='A'), 
                      Example_Individual(fitness=0.6, name='B'), 
                      Example_Individual(fitness=0.7, name='C'),
                      Example_Individual(fitness=0.8, name='D'), 
                      Example_Individual(fitness=0.9, name='E')]

print(random_selection(example_population).name)
print(tournament_selection(example_population, 3).name)
print(roulette_selection(example_population).name)
print(rang_selection(example_population).name)

D
A
B
C


# Crossover

In [190]:
def order_crossover(parent1, parent2):
    min_len = min(len(parent1), len(parent2))

    def oc(p1, p2, lb, ub):
        ch = [None for _ in range(min_len)]
        ch[lb:ub] = p1[lb:ub]

        p2_index = ub
        ch_index = ub
        while None in ch:
            if ch_index == len(ch):
                ch_index = 0
            if p2_index == len(p2):
                p2_index = 0
            if p2[p2_index] not in ch:
                ch[ch_index] = p2[p2_index]
                ch_index += 1
            p2_index += 1
        return ch

    lb, ub = sorted(random.sample(range(min_len + 1), 2))

    child1 = oc(parent1, parent2, lb, ub)
    child2 = oc(parent2, parent1, lb, ub)

    return child1, child2

In [191]:
def check_if_visited(routes, individual):
    visited = [False for city in range(individual.num_of_cities + 1)]
    for route in routes:
        for city in route:
            if not visited[city]:
                visited[city] = True
            else:
                print('\tCITY ALREADY VISITED:', city, 'IN ROUTE:', route)

x_child = Individual(input1, 200, 50, 100, 10)
y_child = Individual(input1, 200, 50, 100, 10)

x_child_sol, y_child_sol = order_crossover(x.solution, y.solution)
x_child.solution = x_child_sol
y_child.solution = y_child_sol

print(x.solution)
print(y.solution)
print(x_child_sol)
print(y_child_sol)

check_if_visited(x_child.get_routes(), x_child)
check_if_visited(y_child.get_routes(), y_child)

[80, 93, 85, 43, 13, 58, 101, 23, 75, 89, 37, 91, 100, 102, 32, 11, 50, 26, 1, 70, 103, 59, 15, 73, 22, 56, 4, 77, 104, 62, 94, 99, 6, 25, 105, 92, 95, 3, 79, 34, 35, 106, 5, 20, 9, 66, 107, 72, 7, 46, 17, 60, 108, 42, 84, 61, 55, 109, 27, 28, 69, 88, 18, 10, 48, 110, 45, 44, 86, 97, 96, 111, 33, 12, 76, 81, 54, 24, 112, 14, 57, 87, 68, 113, 39, 21, 40, 53, 74, 114, 83, 30, 51, 115, 2, 98, 16, 116, 63, 64, 90, 117, 31, 52, 19, 8, 118, 47, 49, 119, 36, 120, 82, 78, 121, 65, 71, 122, 29, 67, 123, 38, 41]
[95, 2, 32, 10, 1, 77, 101, 59, 83, 61, 85, 58, 93, 91, 100, 102, 92, 98, 17, 99, 94, 97, 37, 13, 89, 103, 5, 82, 50, 3, 25, 80, 104, 72, 9, 81, 4, 105, 42, 15, 73, 22, 26, 24, 106, 27, 28, 38, 43, 74, 107, 66, 30, 51, 34, 35, 70, 108, 33, 12, 76, 79, 20, 48, 109, 96, 16, 56, 55, 54, 110, 8, 46, 19, 49, 60, 111, 45, 44, 84, 112, 14, 53, 6, 68, 113, 39, 41, 57, 114, 63, 62, 88, 115, 31, 69, 7, 18, 116, 78, 29, 117, 47, 11, 90, 118, 36, 64, 119, 52, 21, 40, 87, 120, 67, 23, 121, 65, 71, 12

In [192]:
def partially_mapped_crossover(parent1, parent2, child1, child2):
    cutoff_1, cutoff_2 = sorted(random.sample(range(len(parent1) + 1), 2))
    
    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)
    
    child1[cutoff_1:cutoff_2] = parent2[cutoff_1:cutoff_2]
    child2[cutoff_1:cutoff_2] = parent1[cutoff_1:cutoff_2]

   

    middle_map = {}
    for a, b in zip (child1[cutoff_1:cutoff_2], child2[cutoff_1:cutoff_2]):
        middle_map[a] = middle_map.get(a, 0) + 1
        middle_map[b] = middle_map.get(b, 0) + 1
        
    mapping_order = []
    for key, value in middle_map.items():
        if value != 2:
            mapping_order.append(key)
            
    def find_all_occurrences(lst, item):
        return [i for i, x in enumerate(lst) if x == item]    
          
    def replace_a_b(x_pos, y, child, cutoff_1, cutoff_2):
        for i in x_pos:
            if i not in range(cutoff_1, cutoff_2):
                child[i] = y
                break 
                   
        return child
        
    def replace_a_b_child(a, a_pos, b, b_pos, child, cutoff_1, cutoff_2):
        if len(a_pos) != 0:
             return replace_a_b(a_pos, b, child, cutoff_1, cutoff_2)
        else:
            return replace_a_b(b_pos, a, child, cutoff_1, cutoff_2)
        
    for i in range(0, len(mapping_order)-1, 2):
        a = mapping_order[i]
        b = mapping_order[i+1]

        a_child1_positions = find_all_occurrences(child1, a)
        a_child2_positions = find_all_occurrences(child2, a)
        
        
        b_child1_positions = find_all_occurrences(child1, b)
        b_child2_positions = find_all_occurrences(child2, b)
        
        child1 = replace_a_b_child(a, a_child1_positions, b, b_child1_positions, child1, cutoff_1, cutoff_2)
        child2 = replace_a_b_child(a, a_child2_positions, b, b_child2_positions, child2, cutoff_1, cutoff_2)

    return child1, child2

[1 | 2 3 4 | 5]  
[5 | 4 3 1 | 2]   

[1 | 4 3 1 | 5]   
[5 | 2 3 4 | 2]   

4-2, 3-3, 1-4

1: 1   
2: 1   
3: 2   
4: 2  
1 -> 2  

[2 | 1 3 4 | 5]    
[5 | 2 3 4 | 1]    

In [193]:
def best_route_better_adjustment_crossover(individual, parent1, parent2, child1, child2):
    # n/2 best from parent1 into first n/2 of child1
    # the rest elements are from parent2 
    # the positions of vehicles are fixed
        
    def create_child(p1, p2, ch1):
        p1_routes = individual.get_routes()
        p1_routes.sort(key = lambda route: individual.route_fitness(route)[0])

        # copy half of the best routes from parent1 to child 1
        best_routes_index = 0
        i = 0
        node_index = 0
        while best_routes_index <= len(p1_routes) / 2:
            node = ch1[node_index]
            if node <= individual.num_of_cities and i < len(p1_routes[best_routes_index]):
                ch1[node_index] = p1_routes[best_routes_index][i]
                i = i + 1
            else:
                best_routes_index = best_routes_index + 1
                i = 0

            node_index = node_index + 1

        # copy elements from parent2 that aren't in child so far
        p2_index = 0
        while node_index < len(ch1):
            node = ch1[node_index]
            if node <= individual.num_of_cities:
                p2_node = p2[p2_index]
                if p2_node > individual.num_of_cities:
                    p2_index = p2_index + 1
                    p2_node = p2[p2_index]
                
                if p2_node not in ch1[:node_index]:
                    ch1[node_index] = p2_node
                p2_index = p2_index + 1

            node_index = node_index + 1
            
        return ch1
    
    child1 = create_child(parent1, parent2, child1)
    child2 = create_child(parent2, parent1, child2)
    return child1, child2

In [194]:
def crossover(param, parent1, parent2, child1, child2):
    if param["individual"] is not None:
        return param["crossover"](param["individual"], parent1, parent2, child1, child2)
    else:
        return param["crossover"](parent1, parent2, child1, child2)

# Mutation

In [195]:
def swap_mutation(individual):
    l, r = sorted(random.sample(range(len(individual)), 2))
    individual[l], individual[r] = individual[r], individual[l]
    
    return individual

In [196]:
def invert_mutation(individual):
    l, r = sorted(random.sample(range(len(individual) + 1), 2))
    chosen = individual[l:r]
    chosen.reverse()
    individual[l:r] = chosen
    
    return individual

In [197]:
def shaking_mutation(individual):
    l, r = sorted(random.sample(range(len(individual) + 1), 2))
    chosen = individual[l:r]
    random.shuffle(chosen)
    individual[l:r] = chosen
    
    return individual

In [198]:
def mutation(param, individual, mutation_prob):
    if random.random() < mutation_prob:
        individual = param(individual)
        
    return individual

# Fix solution


In [199]:
def insertion_based_repair(individual):
    if (individual.is_feasible()):
        print('FESIBLE')
        return individual
        
    routes = individual.get_routes()
 
    unfeasible_routes = []
    for route in routes:
        is_unfeasible, _ = individual.is_route_unfeasible(route)
        if is_unfeasible:
            unfeasible_routes.append(route)
            routes.remove(route)
            
    #print('\n-------NUMBER OF UNFEASIBLE ROUTES------       ', len(unfeasible_routes))
            
    removed_cities_list = []
    for route in unfeasible_routes:        
        route.sort(key = lambda x: (individual.data[x]["ready_time"], individual.data[x]["due_time"]))
        
        #print('\nCITIES AFTER SORTING THE ROUTE', route)
        
        route_before_city_removal = deepcopy(route) 
        
        # find and eliminate unfeasible cities
        while True:
            is_unfeasible, unfeasible_city = individual.is_route_unfeasible(route)
            if not is_unfeasible:
                break
                
            removed_cities_list.append(unfeasible_city)
            route.remove(unfeasible_city)
  
                    
        #print('\nREMAINING CITIES AFTER REMOVING UNFEASIBLE CITIES', route)
        #if route_before_city_removal == route:
            #print('\tSORTED ROUTE BECAME FEASIBLE!')
            
        routes.append(route)
        
    # first try: insert if possible in existing route
    if len(removed_cities_list) > 0:
        #print('\nENTERING FIRST TRY IN REPAIR...................................')
        #print('NUMBER OF UNFEASIBLE CITIES', len(removed_cities_list))
        
        for city_index in removed_cities_list:
            is_inserted = False
            for route_index, route in enumerate(routes):            
                for i in range(len(route)):
                    route_copy = deepcopy(route)
                    route_copy.insert(i, city_index)
                    
                    is_unfeasible, _ = individual.is_route_unfeasible(route_copy)
                    if not is_unfeasible:
                        #print('\n\tADDED A CITY TO A ROUTE SUCCESSFULLY')
                        #print('\tROUTE BEFORE ADDING THE CITY:', routes[route_index])
                        routes[route_index] = route_copy
                        #print('\tROUTE AFTER ADDING THE CITY:', routes[route_index])
                        is_inserted = True
                        removed_cities_list.remove(city_index)
                        break
                        
                if is_inserted:
                    break
                    
    # second try: create new routes 
    if len(removed_cities_list) > 0 and individual.num_of_vehicles - len(routes) > 0:
        #print('\nENTERING SECOND TRY IN REPAIR...................................')
        #print('NUMBER OF UNFEASIBLE CITIES', len(removed_cities_list))
        
        new_routes = [[] for _ in range( individual.num_of_vehicles - len(routes) )]
        generated_routes = individual.generate_feasible_routes(new_routes, removed_cities_list)
        #print('\tGENERATED ROUTES', generated_routes)
        #for new_route in generated_routes:
            #is_unfeasible, _ = individual.is_route_unfeasible(new_route)
            #if is_unfeasible:
                #print('GENERATED ROUTE IS UNFEASIBLE:', new_route)
        #print('GENERATED ROUTES ARE ALL FEASIBLE!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        routes = routes + generated_routes
        #print('\tROUTES + NEW ROUTES\n', routes)
        

    individual.solution = individual.create_solution(routes)
    #print('NEW SOLUTION:\n', individual.solution)
        
    return individual

In [201]:
x = Individual(input1, 200, 50, 100, 10)
y = Individual(input1, 200, 50, 100, 10)

def print_feasibility(x, y):
    print('\t', 'is x feasible:', x.is_feasible())
    print('\t', 'is y feasible:', y.is_feasible())
    print('\n########################################################\n')

def check_if_visited(routes, individual):
    visited = [False for city in range(individual.num_of_cities + 1)]
    for route in routes:
        for city in route:
            if not visited[city]:
                visited[city] = True
            else:
                print('\tCITY ALREADY VISITED:', city, 'IN ROUTE:', route)
    
print('initial individuals..........')

check_if_visited(x.get_routes(), x)
check_if_visited(y.get_routes(), y)
print_feasibility(x, y)
x_solution_previous = x.solution
y_solution_previous = y.solution

print('order crossover.........')
x_child_solution, y_child_solution = order_crossover(x.solution, y.solution)
print('\t', 'same x solutions:', x_child_solution == x_solution_previous)
print('\t', 'same y solutions:', y_child_solution == y_solution_previous)

x_child = Individual(input1, 200, 50, 100, 10)
x_child.solution = x_child_solution
y_child = Individual(input1, 200, 50, 100, 10)
y_child.solution = y_child_solution

check_if_visited(x_child.get_routes(), x_child)
check_if_visited(y_child.get_routes(), y_child)
print_feasibility(x_child, y_child)
x_solution_previous = x_child.solution
y_solution_previous = y_child.solution

print('insertion based repair........')

print('-----------------------X CHILD-------------------------')
x_child = insertion_based_repair(x_child)
check_if_visited(x_child.get_routes(), x_child)

print('-----------------------Y CHILD-------------------------')

y_child = insertion_based_repair(y_child)
check_if_visited(y_child.get_routes(), y_child)

print('\t', 'same x solutions:', x_child.solution == x_solution_previous)
print('\t', 'same y solutions:', y_child.solution == y_solution_previous)
print_feasibility(x_child, y_child)

initial individuals..........
	 is x feasible: True
	 is y feasible: True

########################################################

order crossover.........
	 same x solutions: False
	 same y solutions: False
	 is x feasible: False
	 is y feasible: False

########################################################

insertion based repair........
-----------------------X CHILD-------------------------
-----------------------Y CHILD-------------------------
	 same x solutions: False
	 same y solutions: False
	 is x feasible: True
	 is y feasible: True

########################################################



# Genetic algorithm

In [127]:
def genetic_algorithm(params):
    
    data = params["data"]
    capacity = params["capacity"]
    num_of_vehicles = params["num_of_vehicles"]
    service_time = params["service_time"]
    population_size = params["population_size"]
    num_generations = params["num_generations"]
    elitism_size = params["elitism_size"]
    tournament_size = params["tournament_size"]
    selection_params = params["selection"]
    crossover_param = params["crossover"]
    mutation_params = params["mutation"]
    mutation_prob = params["mutation_prob"]
    
    population = [Individual(data, capacity, num_of_vehicles, len(data)-1, service_time ) for _ in range(population_size)]
    #print([x.fitness for x in population])
    new_population = deepcopy(population)
  
    crossover_params = { "crossover" : crossover_param, 
                         "individual" : population[0] if crossover_param.__name__ == "best_route_better_adjustment_crossover" else None }
    best_solutions = []
    for i in range(num_generations):
        population.sort(key = lambda x: x.fitness)
        #print([x.fitness for x in population])
        best_solutions.append(population[0])
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1 = selection(selection_params, population)
            parent2 = selection(selection_params, population)
            
            while(parent1 == parent2):
                parent2 = selection(selection_params, population)

            
            new_population[j].solution, new_population[j+1].solution = crossover(crossover_params, 
                                                                                 parent1.solution, 
                                                                                 parent2.solution, 
                                                                                 new_population[j].solution, 
                                                                                 new_population[j+1].solution)
                
            
            new_population[j].solution = mutation(mutation_params, new_population[j].solution, mutation_prob)
            new_population[j+1].solution = mutation(mutation_params, new_population[j+1].solution, mutation_prob)

            insertion_based_repair(new_population[j])
            insertion_based_repair(new_population[j+1])

            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()

        population = deepcopy(new_population)
        
    
    return min(population, key = lambda x: x.fitness), best_solutions

In [ ]:
POPULATION_SIZE = 1000
ELITISIM_SIZE = 100
MUTATION_PROB = 0.1
TOURNAMENT_SIZE = 300
NUM_GENERATIONS = 100
CAPACITY = 300
SELECTION = rang_selection
CROSSOVER = order_crossover
MUTATION = shaking_mutation
NUM_OF_VEHICLES = 50
SERVICE_TIME = 10

params = {
    "data" : input1,
    "population_size" : POPULATION_SIZE,
    "elitism_size" : ELITISIM_SIZE,
    "mutation_prob" : MUTATION_PROB,
    "tournament_size" : TOURNAMENT_SIZE,
    "num_generations" : NUM_GENERATIONS,
    "capacity" : CAPACITY,
    "selection" : { "selection" : SELECTION, 
                    "tournament_size" : TOURNAMENT_SIZE if SELECTION.__name__ == "tournament_selection" else None
                  },
    "crossover" : CROSSOVER,
    "mutation" : MUTATION,
    "num_of_vehicles" : NUM_OF_VEHICLES,
    "service_time" : SERVICE_TIME
}

ga_order_crossover, best_solutions_order_crossover = genetic_algorithm(params)

# params["crossover"] = partially_mapped_crossover

# ga_pmc, best_solutions_pmc = genetic_algorithm(params)

# params["crossover"] = best_route_better_adjustment_crossover

# ga_brbac, best_solutions_brbac = genetic_algorithm(params)


In [ ]:
ga_order_crossover.is_feasible()

In [ ]:
print(sum(1 for route in ga_order_crossover.get_routes() if len(route) > 0))
print(len(ga_order_crossover.get_routes()))
print(ga_order_crossover.fitness)

In [ ]:
ga_pmc.is_feasible()

In [ ]:
ga_brbac.is_feasible()

In [ ]:
ga_brbac.solution == ga_pmc.solution

egzaktan, grafovi, viseciljna, reference, prezentacija i pdf



# Graphs

In [ ]:
def plot_cities(x_coordinates, y_coordinates, dataset_name):
    plt.figure(figsize=(8, 6))
    plt.scatter(x_coordinates[1:], y_coordinates[1:], color='red', label='cities')
    plt.scatter(x_coordinates[0], y_coordinates[0], color='blue', label='Depot') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('City Coordinates for ' + dataset_name)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
x_input1 = [x["x"] for x in input1]
y_input1 = [x["y"] for x in input1]

x_input2 = [x["x"] for x in input2]
y_input2 = [x["y"] for x in input2]

x_input3 = [x["x"] for x in input3]
y_input3 = [x["y"] for x in input3]


plot_cities(x_input1, y_input1, "input_1")


In [ ]:
#Best solution routes 
def plot_best_solution(x, title):
    plt.figure(figsize=(8, 6))
    routes = x.get_routes()
    for route in routes:
        x_points  = np.array([x.depot["x"]])
        x_points = np.append(x_points, [x.data[i]["x"] for i in route], axis = 0)
        x_points = np.append(x_points, [x.depot["x"]], axis = 0)
        y_points  = np.array([x.depot["y"]])
        y_points = np.append(y_points, [x.data[i]["y"] for i in route], axis = 0)
        y_points = np.append(y_points, [x.depot["y"]], axis = 0)
        

        plt.plot(x_points, y_points)
    
    plt.title(title)
    plt.show


In [ ]:
plot_best_solution(ga_order_crossover, "Best Solution Routes OC")
plot_best_solution(ga_pmc, "Best Solution Routes PMC")
plot_best_solution(ga_brbac, "Best Solution Routes BRBAC") 

In [ ]:
def best_solution_evolution(solutions, title, info):
    plt.figure(figsize=(8, 6))
    y_points = [[] for _ in range(len(solutions))]
    i = 0
    for solution in solutions:
        y_points[i] = [x.fitness for x in solution]
        i+=1
    
    for i,y in enumerate(y_points):
        plt.plot(np.array(y), label = info[i])
    plt.title(title)
    plt.ylabel("fitness")
    plt.xlabel("generation")
    plt.legend()
    plt.show()

In [ ]:
best_solution_evolution([best_solutions_order_crossover,best_solutions_brbac, best_solutions_pmc], "The Effect Of Crossover Function On Fitness Using Nearest Neighbour Initialization", ["Order Crossover", "Best Route Better Adjustment Crossover", "Partially Mapped Crossover"])